# Setup and Validation

This notebook validates your environment and configuration before running the SLAVV comparison pipeline. It replaces the legacy `validate_setup.py` script.

## ⚠️ Environment Check

**IMPORTANT**: Before proceeding, ensure you are using a properly configured virtual environment.

### Quick Check
Run the cell below. If you see errors, follow the setup guide at `docs/guides/SETUP.md`.

In [1]:
import sys
from pathlib import Path

print(f"Python Executable: {sys.executable}")
print(f"Python Version: {sys.version}")

# Check if we're in a virtual environment
in_venv = hasattr(sys, 'real_prefix') or (hasattr(sys, 'base_prefix') and sys.base_prefix != sys.prefix)
if in_venv:
    print("✅ Running in a virtual environment")
else:
    print("⚠️ NOT running in a virtual environment")
    print("   Recommendation: Create a venv or conda environment. See docs/guides/SETUP.md")

# Try importing the package
try:
    import slavv
    print(f"✅ SLAVV package found at: {slavv.__file__}")
except ImportError:
    print("❌ SLAVV package not installed!")
    print("   Run: pip install -e .")
    print("   See docs/guides/SETUP.md for full setup instructions")

In [2]:
%load_ext autoreload
%autoreload 2

import subprocess
import sys
from pathlib import Path

# Import validation module directly from installed package
from slavv.dev.validation import Validator


## Configuration

Set your paths below. If you are unsure, the defaults usually work for standard installations.

In [3]:
# Path to MATLAB executable
# Windows example: "C:\\Program Files\\MATLAB\\R2023a\\bin\\matlab.exe"
import shutil
matlab_path = shutil.which('matlab') or "C:\\Program Files\\MATLAB\\R2019a\\bin\\matlab.exe"
if not shutil.which('matlab') and matlab_path == "C:\\Program Files\\MATLAB\\R2019a\\bin\\matlab.exe":
    print("⚠️ MATLAB not found in PATH. Using default fallback path. Please update 'matlab_path' below if incorrect.")

# Path to test data volume
# Using data from tests/data directory
project_root = Path(slavv.__file__).parent.parent.parent
test_data = project_root / "tests" / "data" / "slavv_test_volume.tif"

print(f"Project Root: {project_root}")
print(f"MATLAB Path: {matlab_path}")
print(f"Test Data: {test_data}")

## Run Validation

Execute the validation checks. This looks for:
1. Required Python packages
2. Valid MATLAB installation
3. Necessary external submodules (Vectorization-Public)
4. Test data integrity

In [4]:
validator = Validator(project_root, matlab_path=matlab_path, test_data_path=str(test_data))

# Run standard checks
validator.check_python_imports()
validator.check_vectorization_public()
validator.check_matlab_executable()
validator.check_test_data()
validator.check_disk_space(project_root / "comparison_output")

# Run specific MATLAB version check (requires launching MATLAB, may take a moment)
print("Checking MATLAB version...")
validator.check_matlab_version()

## MATLAB Functional Test

This runs a small MATLAB script to confirm it can execute code and find the vectorization library.

In [5]:
# Run the robust functional test from the Validator class
print("Running MATLAB functional test...")
validator.check_matlab_functional()

## Summary Results

In [6]:
print("Passed Checks:")
for p in validator.passed:
    print(f"✅ {p.message}")

if validator.warnings:
    print("\nWarnings:")
    for w in validator.warnings:
        print(f"⚠️ {w.message}")

if validator.errors:
    print("\nERRORS:")
    for e in validator.errors:
        print(f"❌ {e.message}")
    print("\n💡 If you see import errors, check docs/guides/SETUP.md")
else:
    print("\n🎉 System is ready for comparison runs!")